In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from IPython.display import clear_output
!pip install pandas tqdm torch scikit-learn
clear_output()

In [6]:
import os
import argparse
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import models
from data.dataset import PatientDataset

In [9]:
images_path = "dataset/reduced/train"
labels_path = "dataset/reduced/train.csv"
batch_size = 32
model = "cnn"
cuda = True
lr = 1e-3
epochs=100
depth = 100 # number of CT slices

In [21]:
entries = os.listdir(images_path)
patient_ids = [int(entry) for entry in entries if os.path.isdir(os.path.join(images_path, entry))]

df = pd.read_csv(labels_path)
df = df[df['patient_id'].isin(patient_ids)]
print(df.shape)

(2886, 6)


In [22]:
# Split the dataset into train, validation, and test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=42)

print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(2308, 6)
(289, 6)
(289, 6)


In [23]:
train_data.head()

,patient_id,bowel,extravastion,kidney,liver,spleen
1426,38343,0,1,0,0,1
964,29412,0,0,0,0,0
2608,63193,0,0,0,0,0
662,23709,0,0,0,0,0
2408,58547,0,0,0,0,0


In [24]:
train_dataset = PatientDataset(images_path, train_data)
val_dataset = PatientDataset(images_path, val_data)
test_dataset = PatientDataset(images_path, test_data)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# get one sample
inputs, labels = train_dataset[0]

print(inputs.shape)
print(labels.shape)

torch.Size([1, 100, 128, 128])
torch.Size([5])


In [25]:

if model == "cnn":
    model = models.ConvNet3D(
        in_channels=inputs.shape[0],
        out_channels=labels.shape[0],
        depth=inputs.shape[1],
        height=inputs.shape[2],
        width=inputs.shape[3],
    )
elif model == "unet":
    model = ...
else:
    raise ValueError("Invalid model selected for training.")

if cuda:
    model = model.cuda()

criterion = nn.BCEWithLogitsLoss()  # multi-label classification
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    running_loss = 0.0

    for i, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs, labels = batch
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero gradients for every batch
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")


  3%|▎         | 20/577 [00:17<08:05,  1.15it/s]


KeyboardInterrupt: 